In [12]:
import os, sys
import pandas as pd

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW
reload(RTCW)

db_name = 'rtcw_db.db'
demos_folder = 'demos'
cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
root_path = 'C:\\Users\\Jelle\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [13]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [14]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [15]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
demo_df = db.get_table_as_df(table_name= 'demo')
player_df = db.get_table_as_df(table_name= 'player')
roundstats_df = db.get_table_as_df(table_name='roundstats')
bulletevent_df = db.get_table_as_df(table_name='bulletevent')
obituary_df = RTCW.add_match_data(obituary_df, player_df, demos_dct)
obituary_df = obituary_df.loc[obituary_df['League'] == 'bob1']
#obituary_df = obituary_df.loc[obituary_df['Importance'].isin(['lb-round3', 'wb-semi-finals', 'lb-round4'])]

bulletevent_df = RTCW.add_match_data(bulletevent_df, player_df, demos_dct)
chatmessages_df = db.get_table_as_df(table_name= 'chatmessage')
chatmessages_df = RTCW.add_match_data(chatmessages_df, player_df, demos_dct, what_df = 'chatmessages_df')
chatmessages_df = RTCW.feature_extraction_chat(chatmessages_df)

obituary table fetched in 0.53 seconds
demo table fetched in 0.04 seconds
player table fetched in 0.1 seconds
roundstats table fetched in 0.11 seconds
bulletevent table fetched in 1.14 seconds
chatmessage table fetched in 0.82 seconds


In [41]:
df_spree = RTCW.get_kill_sprees(obituary_df, 
                                demo_df,
                                maxtime_secs = 8, 
                                include_weapon_filter = None,
                                exclude_weapon_filter = ['smg','pistol'], 
                                minspree = 4,
                                pov_sprees_only = False,
                                verbose = True)

all done!


In [42]:
df_spree[['match', 'demo', 'player', 'spreecount', 'weapons', 'attacker', 'pov_id']].sort_values('spreecount', ascending=False)

,match,demo,player,spreecount,weapons,attacker,pov_id
0,rtcw_2002.01.31_bob1_empire_vs_4ku_round1,1012542044demo0000.dm_57,Xguy{4ku},4,46-46-46-46-,6,5
1,rtcw_2002.01.31_bob1_empire_vs_4ku_round1,1012542044demo0000.dm_57,Foo{4ku},4,6-7-7-7-,5,5
2,rtcw_2002.01.29_bob1_tdp_vs_cp_round1,[cp]dIsRuPtIvE.dm_57,[cp]scope,4,46-46-46-46-,2,9
3,rtcw_2002.01.31_bob1_molan_vs_tbar_round1,tbirdkiri.dm_57,[mol]pills,4,32-32-32-32-,10,11
4,rtcw_2002.02.12_bob1_s62_vs_flying-hellfish_ro...,S62Gungadi.dm_57,S62gungadi,4,7-7-7-7-,3,3
5,rtcw_2002.01.29_bob1_sin_vs_dpg_round1,1012371601demo01drmack.dm_57,Panzerkunst-dpg,4,6-7-7-5-,10,4
6,rtcw_2002.01.31_bob1_wolfmorder_vs_cya_round1,Avenger.dm_57,-wm-idiotsavant-,4,6-5-5-5-,3,6
7,rtcw_2002.02.07_bob1_narf_vs_abuse_round2,1013146769Demo_01.dm_57,Acujo,4,46-46-46-46-,8,14
8,rtcw_2002.01.29_bob1_ctw_vs_portuguese_round1,1012415192Canadian_Throwing_Wangs_vs_The_Portu...,Ocean_nb,4,3-3-3-3-,13,11
9,rtcw_2002.01.29_bob1_hv_vs_vex_round1,1012368243HV-vup-beach-round1.dm_57,Carnage_inchv,4,6-7-7-7-,10,4


In [43]:
RTCW.cut_demos(root_path, demos_dct, df_spree, offset_start = 8, offset_end = 8, transform_to_dm_60 = True, demo_folder_name= 'demos')
RTCW.generate_capture_list(df_spree,  transform_to_dm_60 = True)

In [ ]:
for demo in df_spree.md5:
    for k in demos_dct:
        if demo in demos_dct[k][0]:
            print k

In [23]:
docs_df = RTCW.get_docruns(chatmessages_df, min_docrun_length = 10, max_timeleft = 30, docs_succesful = True, min_docs_lost = 0, verbose = True)

C:\Users\Jelle\Documents\GIT\rtcw_demo_analyzer\src\rtcw_functions.py:733: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  arr = df_demo.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime', 'SecondsLeftInRound'])


scanned 100 demos of 731 demos in total
scanned 200 demos of 731 demos in total
scanned 300 demos of 731 demos in total
scanned 400 demos of 731 demos in total
scanned 500 demos of 731 demos in total
scanned 600 demos of 731 demos in total
scanned 700 demos of 731 demos in total
all done!


In [24]:
save_index = docs_df[['start', 'end', 'start_secsleft', 'end_secsleft', 'duration', 'times_lost_docs', 'won_round']].drop_duplicates().index
docs_df = docs_df[docs_df.index.isin(save_index)]

In [25]:
docs_df[['match','start', 'end', 'start_secsleft', 'end_secsleft', 'duration', 'times_lost_docs', 'won_round']]

,match,start,end,start_secsleft,end_secsleft,duration,times_lost_docs,won_round
88,rtcw_2003.08.15_qcon03_amish_vs_ck_lb-round1,18881250,18897500,28,12,16,1,True
405,rtcw_2003.08.15_qcon03_narf_vs_ck_lb-round2,8022900,8036500,30,17,13,0,True
609,rtcw_2003.08.15_qcon03_narf_vs_wsw_wb-round2,25200500,25234700,45,10,35,0,True
940,rtcw_2003.08.14_qcon03_dh-eu_vs_icf_wb-round1,9851000,9884250,61,27,34,0,True
1003,rtcw_2003.08.14_qcon03_4kings_vs_wsw_wb-round1,7009900,7044650,50,15,35,0,True
1225,rtcw_2003.08.15_qcon03_infensus_vs_wsw_wb-semi...,60138550,60211500,77,4,73,1,True
1267,rtcw_2003.08.15_qcon03_gmpo_vs_hv_lb-round1,17555100,17613100,77,19,58,1,True


In [26]:
reload(RTCW)
RTCW.cut_demos(root_path, demos_dct, docs_df, demo_type = 'docs', offset_start = 8, offset_end = 8, transform_to_dm_60 = True, demo_folder_name= 'demos')

In [ ]:
RTCW.generate_capture_list(docs_df, demo_type='docs', transform_to_dm_60 = True)